In [1]:
from timelapse_tools import compute
from timelapse_tools.compute.min_max_intensity_finder import MinMaxIntensityFinder

In [2]:
# Initialize prior to passing in
# This is how you can parametrize your computation manager
computation_manager = MinMaxIntensityFinder()

In [3]:
results = compute("timelapse_tools/tests/data/s_1_t_5_c_1_z_1.czi", computation_manager, show_progress=True)
(results.min_intensity, results.max_intensity)

 20%|██        | 1/5 [00:00<00:00,  6.76it/s]

/Users/jacksonb/Desktop/active/cell/timelapse_access/timelapse_tools/tests/data/s_1_t_5_c_1_z_1.czi


100%|██████████| 5/5 [00:00<00:00,  7.08it/s]


(347, 1894)

In [4]:
# Don't worry, even though this produces the same min and max
# The size of T is 1 for the data cube that reaches computation_manager.process_data function
results = compute("timelapse_tools/tests/data/s_1_t_5_c_1_z_1.czi", computation_manager, operating_dimension="Z", T=0, show_progress=True)
(results.min_intensity, results.max_intensity)

100%|██████████| 1/1 [00:00<00:00,  6.87it/s]

/Users/jacksonb/Desktop/active/cell/timelapse_access/timelapse_tools/tests/data/s_1_t_5_c_1_z_1.czi


(347, 1894)